# VPR - Vehicle Routing Problem

## What you should have by now:

### FROM Notebook Create Graph

- nodes_final.csv - Contains all nodes of the city graph. The city graph is big enough to enclose all warehouses in the city
- edges_final.csv - The edges connecting the nodes including columns like 'gh', 'lat/lng', 'type', 'maxspeed', 'dist', 'realspeed',  'drive_time'
- a dicts folder inside the output (currently named new sol) folder. These dictionaries contain the drive times from every node to every other node (at the moment 30000^2 ~= 10^8 key value pairs)
- where_nodes_dict.csv (inside dict folder) - This dictionary contains a complete mapping of which node and its corresponding drive times is saved in which dict file.

### FROM Notebook Create Graph

- scooters_final.csv - Dataframe containing the scooters matched to the nodes with the columns: node_id, scooter_ids (SOMETIMES MULTIPLE SCOOTERS PER NODE), num_scooters (= number of scooters)

In [1]:
# https://developers.google.com/optimization/routing/vrp

In [2]:
# VRP
from __future__ import print_function

import pandas as pd
import numpy as np
import json
import os
import random
import overpy
import pprint
import geojson
import time
import pickle
from h3 import h3
from keplergl import KeplerGl
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm
from datetime import timedelta

# VRP
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [3]:
def try_create_folder(folder_name: str):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    return None


def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = (np.sin(dlat / 2) ** 2
         + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2)
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km * 1000

def apply_hav(x):
    try:
        return haversine(float(x.start_lng), float(x.start_lat), 
                         float(x.end_lng), float(x.end_lat))
    except:
        print(x.start_lng, x.start_lat, x.end_lng, x.end_lat)
        return None

## Parameters

In [4]:
inputs_folder = 'inputs'
output_folder = 'new_sol'
dict_store = 'dicts'
plots_path = os.path.join(output_folder, 'plots')
dicts_path = os.path.join(output_folder, dict_store)
try_create_folder(output_folder)
try_create_folder(plots_path)
try_create_folder(dicts_path)

wh_df = pd.DataFrame([
    ['wh0', 52.436, 13.376],
    ['wh1', 52.561, 13.475]
], columns=['name', 'lat', 'lng'])

city_lat, city_lng = 52.52, 13.40
len_half_window = 75
reasonable_radius_margin = 0.05

realistic_speed = {
    3: 3,
    5: 5,
    6: 6,
    7: 7,
    10: 8,
    15: 13,
    17: 15,
    20: 18,
    30: 25,
    40: 33,
    50: 37,
    60: 54,
    70: 62,
    80: 71,
    100: 90,
    130: 120
}

time_penalty_per_scooter = 60*3

# Berlin Bounding Box
# NW 52.6716, 13.0875
# SO 52.3923, 13.6858

## VPR

In [5]:
"""
# TEST DATA TO TEST solve_routing_problem()
# RUN WITH:
data = create_data_model()
routes_dict = solve_routing_problem(data, True)

def create_data_model():
    # Stores the data for the problem.
    data = {}
    data['time_matrix'] = [
      [0, 6, 9, 8, 7, 3, 6, 2, 3, 2, 6, 6, 4, 4, 5, 9, 7],
      [6, 0, 8, 3, 2, 6, 8, 4, 8, 8, 13, 7, 5, 8, 12, 10, 14],
      [9, 8, 0, 11, 10, 6, 3, 9, 5, 8, 4, 15, 14, 13, 9, 18, 9],
      [8, 3, 11, 0, 1, 7, 10, 6, 10, 10, 14, 6, 7, 9, 14, 6, 16],
      [7, 2, 10, 1, 0, 6, 9, 4, 8, 9, 13, 4, 6, 8, 12, 8, 14],
      [3, 6, 6, 7, 6, 0, 2, 3, 2, 2, 7, 9, 7, 7, 6, 12, 8],
      [6, 8, 3, 10, 9, 2, 0, 6, 2, 5, 4, 12, 10, 10, 6, 15, 5],
      [2, 4, 9, 6, 4, 3, 6, 0, 4, 4, 8, 5, 4, 3, 7, 8, 10],
      [3, 8, 5, 10, 8, 2, 2, 4, 0, 3, 4, 9, 8, 7, 3, 13, 6],
      [2, 8, 8, 10, 9, 2, 5, 4, 3, 0, 4, 6, 5, 4, 3, 9, 5],
      [6, 13, 4, 14, 13, 7, 4, 8, 4, 4, 0, 10, 9, 8, 4, 13, 4],
      [6, 7, 15, 6, 4, 9, 12, 5, 9, 6, 10, 0, 1, 3, 7, 3, 10],
      [4, 5, 14, 7, 6, 7, 10, 4, 8, 5, 9, 1, 0, 2, 6, 4, 8],
      [4, 8, 13, 9, 8, 7, 10, 3, 7, 4, 8, 3, 2, 0, 4, 5, 6],
      [5, 12, 9, 14, 12, 6, 6, 7, 3, 3, 4, 7, 6, 4, 0, 9, 2],
      [9, 10, 18, 6, 8, 12, 15, 8, 13, 9, 13, 3, 4, 5, 9, 0, 9],
      [7, 14, 9, 16, 14, 8, 5, 10, 6, 5, 4, 10, 8, 6, 2, 9, 0],
    ]
    data['demands'] = [0, 1, 1, 3, 6, 3, 6, 8, 8, 1, 2, 1, 2, 6, 6, 8, 8]
    data['vehicle_capacities'] = [5, 10, 15, 15]
    data['vehicle_max_working_time'] = [10, 20, 30, 10]
    data['start'] = [0, 0, 0, 1]
    data['end'] = [0, 0, 0, 1]
    data['num_vehicles'] = 4
    data['penalty_per_scooter'] = 60*3
    return data
"""

print("merry christmas and a happy new year")

merry christmas and a happy new year


In [6]:
# scooters_final - scooter nodes
# wh_final - warehouse nodes
# dist_dict - distances
# drivers - agents
scooters_final = pd.read_csv(os.path.join(
    output_folder, "scooters_final.csv"), index_col=0)
wh_final = pd.read_csv(os.path.join(
    output_folder, "wh_final.csv"), index_col=0)
f = open(os.path.join(dicts_path, 'dist_dict.csv'), 'rb')
dist_dict = pickle.load(f)
f.close()
drivers = pd.read_csv(os.path.join(
    output_folder, 'drivers.csv'), index_col=0)

In [7]:
# scooters_final = scooters_final[
#     scooters_final['node_id'] != 2812574313]

In [8]:
# get the driver_working_time
drivers = drivers.assign(start_time = pd.to_datetime(drivers['start_time']))
drivers = drivers.assign(end_time = pd.to_datetime(drivers['end_time']))
drivers = drivers.assign(driver_working_time = (
    drivers['end_time']-drivers['start_time']).apply(
    lambda x: x.seconds).values)

# get the number of scooters
num_scooters = []
for num in np.concatenate(
    [wh_final['num_scooters'].values, 
     scooters_final['num_scooters'].values]):
    num_scooters.append(int(num))
assert len(num_scooters) == len(wh_final) + len(scooters_final)


# create a dictionary which maps node ids to scooter ids
node_index_to_id = dict()
node_id_to_index = dict()
i = 0
wh_scooter_node_ids = np.concatenate(
    [wh_final['node_id'].values, 
     scooters_final['node_id'].values])

for node in wh_scooter_node_ids:
    node_index_to_id[i] = node
    node_id_to_index[node] = i
    i += 1
    
assert i == len(wh_final) + len(
    scooters_final) == len(wh_scooter_node_ids)


# get start and end node ids of drivers
name_to_node_id = dict(zip(wh_final['name'], 
                           wh_final['node_id']))
start_ids = []
for item in drivers['start_location']:
    start_ids.append(int(node_id_to_index[name_to_node_id[item]]))
end_ids = []
for item in drivers['end_location']:
    end_ids.append(int(node_id_to_index[name_to_node_id[item]]))
drivers = drivers.assign(start_id = start_ids)
drivers = drivers.assign(end_id = end_ids)


# create time_matrix
first_nope = []
second_nope = []
time_matrix = []
for curr_node in wh_scooter_node_ids:
    time_row = []
    for other_node in wh_scooter_node_ids:
        # check if there is an entry in the dist_dict between curr_node and other_node
        if other_node in dist_dict[curr_node].keys():
            time_row.append(int(dist_dict[curr_node][other_node]))
        else:
            first_nope.append([curr_node, other_node])
            if curr_node in dist_dict[other_node].keys():
                time_row.append(int(dist_dict[other_node][curr_node]))
            else:
                second_nope.append([other_node, curr_node])
                time_row.append(int(666666666))
            
    assert len(time_row) == len(wh_scooter_node_ids)
    time_matrix.append(time_row)
    
assert len(time_matrix) == len(wh_scooter_node_ids)
for row in time_matrix:
    assert len(row) == len(time_matrix)

print("no distances found for %0.2f%% of connections" % (
    100*len(first_nope)/len(wh_scooter_node_ids)**2))

if len(first_nope) != 0:
    unique, counts = np.unique(np.array(first_nope)[:,0], 
                               return_counts=True)
    no_dists_stats = pd.DataFrame(np.asarray(
        (unique, counts)).T, columns=['node_id', 'counts'])
    no_dists_stats.sort_values(
        'counts', ascending=False).head()
    
pd.DataFrame(time_matrix).to_csv(
    os.path.join(output_folder, 'time_matrix.csv'))

no distances found for 0.36% of connections


In [11]:
def print_row(row):
    s = ""
    for item in row:
        x = str(item)
        if len(x) == 1:
            s += "%s    " % x
        elif len(x) == 2:
            s += "%s   " % x
        elif len(x) == 3:
            s += "%s  " % x
        else:
            s += "%s " % x
    print(s)

def pretty_sec(seconds):
    return str(timedelta(seconds=seconds))
    

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    # Display dropped nodes.
    dropped_nodes = 'Dropped nodes:'
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node:
            dropped_nodes += ' {}'.format(manager.IndexToNode(node))
    print(dropped_nodes)

    # Display routes
    total_time = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_time = 0
        route_load = 0

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' Node({0}) Load({1}) {2} -> '.format(node_index, route_load, pretty_sec(route_time))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_time += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' Node({0}) Load({1}) {2}\n'.format(manager.IndexToNode(index),
                                                         route_load, pretty_sec(route_time))
        plan_output += 'Time of the route: {}\n'.format(pretty_sec(route_time))
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_time += route_time
        total_load += route_load

    print('Total Time of all routes: {}'.format(pretty_sec(total_time)))
    print('Total Load of all routes: {}'.format(total_load))
    print('Total Possible Load: {}'.format(sum(data['demands'])))
    print('Total Vehicle Capacity: {}'.format(sum(data['vehicle_capacities'])))


def get_routes_dict(data, manager, routing, solution):
    routes_dict = dict()
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route_time = 0
        route_total_load = 0
        route_nodes = []
        route_loads = []

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_nodes.append(node_index)
            route_total_load += data['demands'][node_index]
            route_loads.append(data['demands'][node_index])
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_time += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        node_index = manager.IndexToNode(index)
        route_nodes.append(node_index)
        route_loads.append(data['demands'][node_index])
        routes_dict[vehicle_id] = dict()
        routes_dict[vehicle_id]['time'] = route_time
        routes_dict[vehicle_id]['total_load'] = route_total_load
        routes_dict[vehicle_id]['nodes'] = route_nodes
        routes_dict[vehicle_id]['loads'] = route_loads

    return routes_dict


def solve_routing_problem(data, verbose=False):
    assert len(data['vehicle_capacities']) ==\
           len(data['vehicle_max_working_time']) ==\
           len(data['start']) ==\
           len(data['end'])

    # Create the routing index manager.
    print(len(data['time_matrix']), data['num_vehicles'],
        data['start'], data['end'])
    manager = pywrapcp.RoutingIndexManager(
        len(data['time_matrix']), data['num_vehicles'],
        data['start'], data['end'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(
        time_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(
        transit_callback_index)

    # Add Time constraint.
    time_name = 'Time'
    routing.AddDimension(
        transit_callback_index,
        30*60,  # allow waiting time
        30*60,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time_name)
    time_dimension = routing.GetDimensionOrDie(time_name)
    # time_dimension.SetGlobalSpanCostCoefficient(100)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Allow to drop nodes.
    # Different time penalties for different number of scooters at node
    penalty = data['penalty_per_scooter']
    for idx, node in enumerate(range(1, len(data['time_matrix']))):
        routing.AddDisjunction([manager.NodeToIndex(node)], 
                               penalty*data['demands'][idx])


    # Set working time of van (max time it can drive)
    duration_dimension = routing.GetDimensionOrDie("Time")
    for idx in range(len(data['vehicle_max_working_time'])):
        duration_dimension.CumulVar(routing.End(idx)).SetMax(
            int(data['vehicle_max_working_time'][idx]))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

#     return solution
    if solution:
        if verbose:
            print_solution(data, manager, routing, solution)
        routes_dict = get_routes_dict(data, manager, routing, solution)
        return routes_dict
    else:
        assert 0 == 1, 'something broke'


def create_data_model(
    drivers, time_matrix, num_scooters,
    time_penalty_per_scooter, start_ids, end_ids):
    # Stores the data for the problem.
    data = {}
    data['time_matrix'] = time_matrix
    data['demands'] = num_scooters

    data['vehicle_capacities'] = list(
        drivers['capacity'].values)
    data['vehicle_max_working_time'] = list(
        drivers['driver_working_time'].values)
    data['start'] = start_ids
    data['end'] = end_ids
    data['num_vehicles'] = len(drivers)

    data['penalty_per_scooter'] = time_penalty_per_scooter
    return data


print('start calc routes')
start_time = time.time()

data =  create_data_model(drivers, time_matrix, num_scooters,
                          time_penalty_per_scooter, start_ids, end_ids)
assert len(data['time_matrix']) == len(data['demands'])

if len(data['time_matrix']) > 20:
    print("data['time_matrix'] and data['demands'] too big to print")
else:
    print("data['time_matrix']:")
    [print_row(row) for row in data['time_matrix']];
    print("data['demands']: %s" % (
        str(data['demands'])))
print("data['vehicle_capacities']: %s" % (
    str(data['vehicle_capacities'])))
print("data['vehicle_max_working_time']: %s" % (
    str(data['vehicle_max_working_time'])))
print("data['start']: %s" % (
    str(data['start'])))
print("data['end']: %s" % (
    str(data['end'])))
print("data['num_vehicles']: %s" % (
    str(data['num_vehicles'])))
print("data['penalty_per_scooter']: %s" % (
    str(data['penalty_per_scooter'])))

routes_dict = solve_routing_problem(data, True)

print('calc routes %0.2f sec' % (
    time.time()-start_time))


f = open(os.path.join(output_folder, 'routes_dict.json'), "wb")
pickle.dump(routes_dict, f)
f.close()

start calc routes
data['time_matrix'] and data['demands'] too big to print
data['vehicle_capacities']: [45, 10, 25, 50]
data['vehicle_max_working_time']: [21600, 14400, 14400, 28800]
data['start']: [545, 545, 545, 548]
data['end']: [545, 545, 545, 548]
data['num_vehicles']: 4
data['penalty_per_scooter']: 180
555 4 [545, 545, 545, 548] [545, 545, 545, 548]
Dropped nodes: 2 3 4 5 6 7 8 9 10 12 14 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 57 58 59 61 62 63 64 65 66 67 68 69 70 72 73 74 75 76 77 78 79 80 81 84 88 92 93 97 99 101 102 103 104 105 106 108 109 110 112 114 115 116 118 119 121 122 123 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 144 147 148 149 150 151 153 154 155 156 157 158 159 160 163 164 165 166 167 168 169 170 171 172 173 175 177 178 179 180 181 182 183 186 187 189 190 191 192 195 196 197 198 199 200 201 202 203 205 207 208 209 210 211 212 213 214 215 217 219 220 221 222 223 225 

In [10]:
#     data['time_matrix'] = np.array([
#       [0, 6, 9, 8, 7, 3, 6, 2, 3, 2, 6, 6, 4, 4, 5, 9, 7],
#       [6, 0, 8, 3, 2, 6, 8, 4, 8, 8, 13, 7, 5, 8, 12, 10, 14],
#       [9, 8, 0, 11, 10, 6, 3, 9, 5, 8, 4, 15, 14, 13, 9, 18, 9],
#       [8, 3, 11, 0, 1, 7, 10, 6, 10, 10, 14, 6, 7, 9, 14, 6, 16],
#       [7, 2, 10, 1, 0, 6, 9, 4, 8, 9, 13, 4, 6, 8, 12, 8, 14],
#       [3, 6, 6, 7, 6, 0, 2, 3, 2, 2, 7, 9, 7, 7, 6, 12, 8],
#       [6, 8, 3, 10, 9, 2, 0, 6, 2, 5, 4, 12, 10, 10, 6, 15, 5],
#       [2, 4, 9, 6, 4, 3, 6, 0, 4, 4, 8, 5, 4, 3, 7, 8, 10],
#       [3, 8, 5, 10, 8, 2, 2, 4, 0, 3, 4, 9, 8, 7, 3, 13, 6],
#       [2, 8, 8, 10, 9, 2, 5, 4, 3, 0, 4, 6, 5, 4, 3, 9, 5],
#       [6, 13, 4, 14, 13, 7, 4, 8, 4, 4, 0, 10, 9, 8, 4, 13, 4],
#       [6, 7, 15, 6, 4, 9, 12, 5, 9, 6, 10, 0, 1, 3, 7, 3, 10],
#       [4, 5, 14, 7, 6, 7, 10, 4, 8, 5, 9, 1, 0, 2, 6, 4, 8],
#       [4, 8, 13, 9, 8, 7, 10, 3, 7, 4, 8, 3, 2, 0, 4, 5, 6],
#       [5, 12, 9, 14, 12, 6, 6, 7, 3, 3, 4, 7, 6, 4, 0, 9, 2],
#       [9, 10, 18, 6, 8, 12, 15, 8, 13, 9, 13, 3, 4, 5, 9, 0, 9],
#       [7, 14, 9, 16, 14, 8, 5, 10, 6, 5, 4, 10, 8, 6, 2, 9, 0],
#     ])*30.2435
#     data['demands'] = [0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1]


#     data['time_matrix'] = [
#         [  0, 513, 335, 323, 324, 312, 460, 291, 344, 322, 378, 360, 332, 318, 257, 429, 298, 267, 294, 265, 513],
#         [513,   0, 219, 213, 220, 229, 418, 306, 228, 314, 211, 239, 281, 247, 356, 158, 349, 342, 315, 369, 66666],
#         [335, 219,   0,  13,  19,  84, 326, 106,  53, 113, 105,  83,  81, 47, 156, 121, 149, 142, 116, 171, 219],
#         [323, 213,  13,   0,  14,  79, 314,  93,  48, 101,  94,  71,  69, 34, 144, 116, 137, 130, 104, 159, 213],
#         [324, 220,  19,  14,   0,  76, 318,  97,  54, 105,  98,  74,  73, 38, 147, 122, 140, 133, 107, 162, 220],
#         [312, 229,  84,  79,  76,   0, 386, 125,  99, 159, 149, 134, 140, 106, 179, 155, 181, 165, 128, 169, 229],
#         [460, 418, 326, 314, 318, 386,   0, 314, 288, 264, 279, 273, 291, 289, 304, 285, 257, 290, 328, 327, 418],
#         [291, 306, 106,  93,  97, 125, 314,   0, 113,  57, 139, 127,  80, 81,  55, 202,  56,  41,  22,  65, 306],
#         [344, 228,  53,  48,  54,  99, 288, 113,   0, 105,  69,  35,  52, 35, 164, 118, 154, 150, 123, 178, 228],
#         [322, 314, 113, 101, 105, 159, 264,  57, 105,   0, 128, 119,  52, 77,  69, 191,  51,  55,  73,  85, 314],
#         [378, 211, 105,  94,  98, 149, 279, 139,  69, 128,   0,  35,  85, 70, 190,  67, 174, 175, 149, 204, 211],
#         [360, 239,  83,  71,  74, 134, 273, 127,  35, 119,  35,   0,  66, 51, 178,  95, 167, 163, 137, 192, 239],
#         [332, 281,  81,  69,  73, 140, 291,  80,  52,  52,  85,  66,   0, 42, 118, 148, 101, 104,  90, 135, 281],
#         [318, 247,  47,  34,  38, 106, 289,  81,  35,  77,  70,  51,  42, 0, 131, 133, 124, 117,  91, 146, 247],
#         [257, 356, 156, 144, 147, 179, 304,  55, 164,  69, 190, 178, 118, 131,   0, 253,  47,  14,  59,  33, 356],
#         [429, 158, 121, 116, 122, 155, 285, 202, 118, 191,  67,  95, 148, 133, 253,   0, 237, 238, 212, 267, 158],
#         [298, 349, 149, 137, 140, 181, 257,  56, 154,  51, 174, 167, 101, 124,  47, 237,   0,  32,  71,  69, 349],
#         [267, 342, 142, 130, 133, 165, 290,  41, 150,  55, 175, 163, 104, 117,  14, 238,  32,   0,  51,  39, 342],
#         [294, 315, 116, 104, 107, 128, 328,  22, 123,  73, 149, 137,  90, 91,  59, 212,  71,  51,   0,  60, 315],
#         [265, 369, 171, 159, 162, 169, 327,  65, 178,  85, 204, 192, 135, 146,  33, 267,  69,  39,  60,   0, 369],
#         [513, 66666, 219, 213, 220, 229, 418, 306, 228, 314, 211, 239, 281, 247, 356, 158, 349, 342, 315, 369,   0]
#      ]
#     data['demands'] = [0, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]